# Explore node db storage with hash index

Write a python function that converts a dataclass object into a dictionary

In [1]:
from dataclasses import dataclass, asdict

@dataclass 
class MyDataClass:
    var1: str
    var2: int


def dataclass_to_dict(data_class_object):
    return asdict(data_class_object)


# usage
dc = MyDataClass("Hello", 123)
print(dataclass_to_dict(dc))  # Output: {'var1': 'Hello', 'var2': 123}

{'var1': 'Hello', 'var2': 123}


Write a python function that converts such a dictionary into a hash.

In [2]:
import hashlib
import json

def dict_to_hash(user_dict):
    # Convert the dictionary to a JSON string
    json_str = json.dumps(user_dict, sort_keys=True)
    
    # Hash the JSON string
    hash_object = hashlib.sha256(json_str.encode())

    # Return the hexadecimal representation of hash
    return hash_object.hexdigest()

# Using the function
user_dict = {"name": "John", "age": 30, "city": "New York"}
print(dict_to_hash(user_dict))  

68fe5a470356e94259b33a3b4090f37bca9451ff0a06b3abaa77e4bd7351eb3a


Write a database application that stores the has value, the corresponding dictionary and the path for a file where additional info for this dictionary is stored. Use the hash value as unique ID.

In [3]:
from sqlalchemy import Column, String, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base


Base = declarative_base()

class DictionaryHash(Base):
    __tablename__ = 'dictionary_hash'

    hash_value = Column(String, primary_key=True)
    dictionary = Column(String)
    file_path = Column(String)

engine = create_engine('sqlite:///dict_hash.db', echo=True)
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

# Now, to delete all entries in a model:

session.query(DictionaryHash).delete()
session.commit()


2024-06-09 09:34:28,014 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,015 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dictionary_hash")
2024-06-09 09:34:28,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-09 09:34:28,016 INFO sqlalchemy.engine.Engine COMMIT
2024-06-09 09:34:28,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,017 INFO sqlalchemy.engine.Engine DELETE FROM dictionary_hash
2024-06-09 09:34:28,017 INFO sqlalchemy.engine.Engine [generated in 0.00020s] ()
2024-06-09 09:34:28,019 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
def add_record(hash_value, dictionary, file_path):
    new_record = DictionaryHash(hash_value=hash_value, dictionary=dictionary, file_path=file_path)
    session.add(new_record)
    session.commit()

# Call the function
user_dict = {"name": "John", "age": 31, "city": "New York"}
add_record(dict_to_hash(user_dict), json.dumps(user_dict), "/path/to/file")

2024-06-09 09:34:28,025 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,026 INFO sqlalchemy.engine.Engine INSERT INTO dictionary_hash (hash_value, dictionary, file_path) VALUES (?, ?, ?)
2024-06-09 09:34:28,026 INFO sqlalchemy.engine.Engine [generated in 0.00025s] ('a65bab10c42b5bc7e7246ecd7226e7c534d2290488bae26b822f91c0e3fc9bd3', '{"name": "John", "age": 31, "city": "New York"}', '/path/to/file')
2024-06-09 09:34:28,027 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
def get_all_records():
    for record in session.query(DictionaryHash).all():
        print("Hash value: {}, Dictionary: {}, File Path: {}".format(
            record.hash_value, record.dictionary, record.file_path))

get_all_records()

2024-06-09 09:34:28,031 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,032 INFO sqlalchemy.engine.Engine SELECT dictionary_hash.hash_value AS dictionary_hash_hash_value, dictionary_hash.dictionary AS dictionary_hash_dictionary, dictionary_hash.file_path AS dictionary_hash_file_path 
FROM dictionary_hash
2024-06-09 09:34:28,032 INFO sqlalchemy.engine.Engine [generated in 0.00025s] ()
Hash value: a65bab10c42b5bc7e7246ecd7226e7c534d2290488bae26b822f91c0e3fc9bd3, Dictionary: {"name": "John", "age": 31, "city": "New York"}, File Path: /path/to/file


#### Write a function that outputs and formats get_all_records as a pandas table.

In [6]:
import pandas as pd

def get_all_records():
    records = []
    for record in session.query(DictionaryHash).all():
        records.append({
            'Hash value': record.hash_value,
            'Dictionary': record.dictionary,
            'File Path': record.file_path
        })
        
    df = pd.DataFrame(records)
    return df

# Call the function
df = get_all_records()
df

2024-06-09 09:34:28,571 INFO sqlalchemy.engine.Engine SELECT dictionary_hash.hash_value AS dictionary_hash_hash_value, dictionary_hash.dictionary AS dictionary_hash_dictionary, dictionary_hash.file_path AS dictionary_hash_file_path 
FROM dictionary_hash
2024-06-09 09:34:28,572 INFO sqlalchemy.engine.Engine [cached since 0.5399s ago] ()


,Hash value,Dictionary,File Path
0,a65bab10c42b5bc7e7246ecd7226e7c534d2290488bae2...,"{""name"": ""John"", ""age"": 31, ""city"": ""New York""}",/path/to/file


#### Use Postgres database

**Note:** Start first the postgres server by calling the postgres app

In [7]:
from sqlalchemy import Column, String, JSON, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

Base = declarative_base()

class DictionaryHash(Base):
    __tablename__ = 'dictionary_hash'

    hash_value = Column(String, primary_key=True)
    dictionary = Column(JSON)
    file_path = Column(String)

engine = create_engine('postgresql://localhost/joerg')
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()


# Now, to delete all entries in a model:

session.query(DictionaryHash).delete()
session.commit()

In [8]:
def add_record(hash_value, dictionary, file_path):
    new_record = DictionaryHash(hash_value=hash_value, dictionary=dictionary, file_path=file_path)
    session.add(new_record)
    session.commit()

# Call the function
user_dict = {"name": "John", "age": 30, "city": "New York"}
add_record(dict_to_hash(user_dict), json.dumps(user_dict), "/path/to/file")

In [9]:
# Call the function
df = get_all_records()
df

,Hash value,Dictionary,File Path
0,68fe5a470356e94259b33a3b4090f37bca9451ff0a06b3...,"{""name"": ""John"", ""age"": 30, ""city"": ""New York""}",/path/to/file


In [10]:
from sqlalchemy import text

def query_key_value(key, value):
    results = session.query(DictionaryHash).filter(DictionaryHash.dictionary[key] == text("'{}'".format(value))).all()

    for result in results:
        print("Hash value: {}, Dictionary: {}, File Path: {}".format(
            result.hash_value, result.dictionary, result.file_path))
        
# Call the function
# query_key_value('name', 'John')

In [11]:
def query_key_value(key, value):
    # Note: `.filter(DictionaryHash.dictionary[key] == value)`
    results = session.query(DictionaryHash).filter(DictionaryHash.dictionary[key] == value).all()
    for result in results:
        print("Hash value: {}, Dictionary: {}, File Path: {}".format(
            result.hash_value, result.dictionary, result.file_path
        ))

# Call the function
# query_key_value('name', 'John')

In [12]:
import psycopg2

def query_key_value(key, value):
    # Create a database connection
    conn = psycopg2.connect(database="joerg", user="joerg", password="", host="localhost", port="5432")

    # Create a new cursor
    cur = conn.cursor()

    # Execute a SQL command
    cur.execute(f"SELECT * FROM dictionary_hash WHERE dictionary->>'{key}' = '{value}';")

    # Fetch all results
    rows = cur.fetchall()

    for row in rows:
        print(row)

    # Close the cursor and connection
    cur.close()
    conn.close()

query_key_value('name', 'John')


In [13]:
conn = psycopg2.connect(database="joerg", user="joerg", password="", host="localhost", port="5432")

# Create a new cursor
cur = conn.cursor()

# Execute a SQL command
# cur.execute(f"SELECT * FROM dictionary_hash WHERE dictionary->>'age' = '30';")
cur.execute(f"SELECT * FROM dictionary_hash WHERE file_path = '/path/to/file';")
# cur.execute(f"SELECT * FROM dictionary_hash WHERE dictionary = {json.dumps(user_dict)};")

# cur.execute("SELECT dictionary from dictionary_hash LIMIT 5;")
#cur.execute("SELECT * FROM dictionary_hash WHERE CAST(dictionary as JSON)->>%s = %s;", ('name', 'John'))


# Fetch all results
rows = cur.fetchall()

for row in rows:
    print(row)

# # Close the cursor and connection
cur.close()
conn.close()

('68fe5a470356e94259b33a3b4090f37bca9451ff0a06b3abaa77e4bd7351eb3a', '{"name": "John", "age": 30, "city": "New York"}', '/path/to/file')


In [14]:
json.dumps(user_dict)

'{"name": "John", "age": 30, "city": "New York"}'

In [15]:
import psycopg2
from psycopg2.extras import Json
from sqlalchemy import create_engine

# connect to the database
conn = psycopg2.connect(database="joerg", user="joerg", password="", host="localhost", port="5432")
cur = conn.cursor()

# # create a table with a JSON column
# cur.execute("""
# CREATE TABLE users (
#     id serial PRIMARY KEY,
#     data json
# );
# """)
# conn.commit()

# insert some data
data = {'name': 'John', 'age': 30, 'city': 'New York'}
cur.execute("INSERT INTO users (data) VALUES (%s)", (Json(data),))
conn.commit()

conn.close()
cur.close()


In [16]:
%%time
conn = psycopg2.connect(database="joerg", user="joerg", password="", host="localhost", port="5432")
cur = conn.cursor()

# query for key-value pairs in the json column
# cur.execute("SELECT * FROM users WHERE data ->> 'city' = 'New York';")
cur.execute("SELECT * FROM users WHERE data ->> 'age' = '30';")
print(cur.fetchall())

conn.close()
cur.close()

[(1, {'name': 'John', 'age': 30, 'city': 'New York'}), (2, {'name': 'John', 'age': 30, 'city': 'New York'}), (8, {'name': 'John', 'age': 30, 'city': 'New York'}), (9, {'name': 'John', 'age': 30, 'city': 'New York'}), (10, {'name': 'John', 'age': 30, 'city': 'New York'}), (11, {'name': 'John', 'age': 30, 'city': 'New York'}), (12, {'name': 'John', 'age': 30, 'city': 'New York'}), (14, {'name': 'John', 'age': 30, 'city': 'New York'}), (15, {'name': 'John', 'age': 30, 'city': 'New York'}), (17, {'name': 'John', 'age': 30, 'city': 'New York'}), (18, {'name': 'John', 'age': 30, 'city': 'New York'}), (19, {'name': 'John', 'age': 30, 'city': 'New York'}), (20, {'name': 'John', 'age': 30, 'city': 'New York'}), (22, {'name': 'John', 'age': 30, 'city': 'New York'}), (23, {'name': 'John', 'age': 30, 'city': 'New York'}), (24, {'name': 'John', 'age': 30, 'city': 'New York'}), (25, {'name': 'John', 'age': 30, 'city': 'New York'}), (27, {'name': 'John', 'age': 30, 'city': 'New York'}), (28, {'name':

In [17]:
# error message: AttributeError: Neither 'BinaryExpression' object nor 'Comparator' object has an attribute 'astext'

# from sqlalchemy import create_engine, Column, Integer, String, JSON
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import sessionmaker

# engine = create_engine('postgresql://localhost/joerg', echo=True)

# Base = declarative_base()

# class User(Base):
#     __tablename__ = 'users'

#     id = Column(Integer, primary_key=True)
#     data = Column(JSON)

# # Create table
# Base.metadata.create_all(engine)

# # Insert record
# Session = sessionmaker(bind=engine)
# session = Session()

# new_user = User(data={'name': 'John', 'age': 30, 'city': 'New York'})
# session.add(new_user)
# session.commit()

# # Query JSON field
# users_ny = session.query(User).filter(User.data['city'].astext == 'New York').all()

# for user in users_ny:
#     print(user.data)

# session.close()

In [18]:
# does not work: returns zero matches

from sqlalchemy import create_engine, Column, Integer, JSON
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.sql.expression import cast
from sqlalchemy import String

engine = create_engine('postgresql://localhost/joerg', echo=True)

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    data = Column(JSON)

Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

new_user = User(data={'name': 'John', 'age': 31, 'city': 'New York'})
session.add(new_user)
session.commit()

# Query JSON field
users_ny = session.query(User).filter(cast(User.data['city'], String) == 'New York').all()

print ('num results: ', len(users_ny))
for user in users_ny:
    print('USER DATA: ', user.data)

session.close()


2024-06-09 09:34:28,928 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-06-09 09:34:28,929 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-09 09:34:28,930 INFO sqlalchemy.engine.Engine select current_schema()
2024-06-09 09:34:28,931 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-09 09:34:28,931 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-06-09 09:34:28,932 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-09 09:34:28,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,934 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [19]:
# does not work: returns zero matches

from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker

# Insert record
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)
session = Session()

new_user = User(data={'name': 'John', 'age': 30, 'city': 'New York'})
session.add(new_user)
session.commit()

# Query JSON field
users_ny = session.query(User).filter(cast(User.data['city'], String) == 'New York').all()

for user in users_ny:
    print(user.data)

Session.remove()

2024-06-09 09:34:28,951 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,952 INFO sqlalchemy.engine.Engine INSERT INTO users (data) VALUES (%(data)s) RETURNING users.id
2024-06-09 09:34:28,953 INFO sqlalchemy.engine.Engine [cached since 0.01218s ago] {'data': '{"name": "John", "age": 30, "city": "New York"}'}
2024-06-09 09:34:28,953 INFO sqlalchemy.engine.Engine COMMIT
2024-06-09 09:34:28,954 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,955 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.data AS users_data 
FROM users 
WHERE CAST((users.data -> %(data_1)s) AS VARCHAR) = %(param_1)s
2024-06-09 09:34:28,955 INFO sqlalchemy.engine.Engine [cached since 0.009874s ago] {'data_1': 'city', 'param_1': 'New York'}
2024-06-09 09:34:28,956 INFO sqlalchemy.engine.Engine ROLLBACK


#### SQL query in json column (working!)

In [20]:
from sqlalchemy import text

# Following the same connection and setup steps as before...

# Insert record (same as before)
new_user = User(data={'name': 'John', 'age': 30, 'city': 'New York'})
session.add(new_user)
session.commit()

# Query JSON field using a PostgreSQL JSON function
query = text("SELECT * FROM users WHERE data ->> 'city' = :city")
result = session.execute(query, {'city': 'New York'})

for row in result:
    print(row)

session.close()

2024-06-09 09:34:28,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,961 INFO sqlalchemy.engine.Engine INSERT INTO users (data) VALUES (%(data)s) RETURNING users.id
2024-06-09 09:34:28,961 INFO sqlalchemy.engine.Engine [cached since 0.02075s ago] {'data': '{"name": "John", "age": 30, "city": "New York"}'}
2024-06-09 09:34:28,962 INFO sqlalchemy.engine.Engine COMMIT
2024-06-09 09:34:28,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,964 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE data ->> 'city' = %(city)s
2024-06-09 09:34:28,964 INFO sqlalchemy.engine.Engine [generated in 0.00022s] {'city': 'New York'}
(1, {'name': 'John', 'age': 30, 'city': 'New York'})
(2, {'name': 'John', 'age': 30, 'city': 'New York'})
(3, {'name': 'John', 'age': 31, 'city': 'New York'})
(4, {'name': 'John', 'age': 31, 'city': 'New York'})
(5, {'name': 'John', 'age': 31, 'city': 'New York'})
(6, {'name': 'John', 'age': 31, 'city': 'New York'})
(7, {'name': 'J

In [21]:
from sqlalchemy import text

# Following the same connection and setup steps as before...

# Insert record (same as before)
new_user = User(data={'name': 'John', 'age': 30, 'city': 'New York'})
session.add(new_user)
session.commit()

# Query JSON field using a PostgreSQL JSON function
query = text("SELECT * FROM users WHERE data ->> 'city' = :city")
result = session.execute(query, {'city': 'New York'})

for row in result:
    print(row)

session.close()

2024-06-09 09:34:28,969 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,969 INFO sqlalchemy.engine.Engine INSERT INTO users (data) VALUES (%(data)s) RETURNING users.id
2024-06-09 09:34:28,970 INFO sqlalchemy.engine.Engine [cached since 0.0293s ago] {'data': '{"name": "John", "age": 30, "city": "New York"}'}
2024-06-09 09:34:28,970 INFO sqlalchemy.engine.Engine COMMIT
2024-06-09 09:34:28,971 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,971 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE data ->> 'city' = %(city)s
2024-06-09 09:34:28,971 INFO sqlalchemy.engine.Engine [cached since 0.007502s ago] {'city': 'New York'}
(1, {'name': 'John', 'age': 30, 'city': 'New York'})
(2, {'name': 'John', 'age': 30, 'city': 'New York'})
(3, {'name': 'John', 'age': 31, 'city': 'New York'})
(4, {'name': 'John', 'age': 31, 'city': 'New York'})
(5, {'name': 'John', 'age': 31, 'city': 'New York'})
(6, {'name': 'John', 'age': 31, 'city': 'New York'})
(7, {'name'

In [22]:
%%time
query = text("SELECT * FROM users WHERE data ->> 'city' = :city")
result = session.execute(query, {'city': 'New York'})

for row in result:
    print(row)

session.close()

2024-06-09 09:34:28,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,979 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE data ->> 'city' = %(city)s
2024-06-09 09:34:28,979 INFO sqlalchemy.engine.Engine [cached since 0.01538s ago] {'city': 'New York'}
(1, {'name': 'John', 'age': 30, 'city': 'New York'})
(2, {'name': 'John', 'age': 30, 'city': 'New York'})
(3, {'name': 'John', 'age': 31, 'city': 'New York'})
(4, {'name': 'John', 'age': 31, 'city': 'New York'})
(5, {'name': 'John', 'age': 31, 'city': 'New York'})
(6, {'name': 'John', 'age': 31, 'city': 'New York'})
(7, {'name': 'John', 'age': 31, 'city': 'New York'})
(8, {'name': 'John', 'age': 30, 'city': 'New York'})
(9, {'name': 'John', 'age': 30, 'city': 'New York'})
(10, {'name': 'John', 'age': 30, 'city': 'New York'})
(11, {'name': 'John', 'age': 30, 'city': 'New York'})
(12, {'name': 'John', 'age': 30, 'city': 'New York'})
(13, {'name': 'John', 'age': 31, 'city': 'New York'})
(14, {'name': 'John', 'age

In [23]:
%%time
query = text("SELECT * FROM users WHERE data ->> 'age' = :age")
result = session.execute(query, {'age': '30'})

for row in result:
    print(row)

session.close()

2024-06-09 09:34:28,984 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:28,984 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE data ->> 'age' = %(age)s
2024-06-09 09:34:28,985 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {'age': '30'}
(1, {'name': 'John', 'age': 30, 'city': 'New York'})
(2, {'name': 'John', 'age': 30, 'city': 'New York'})
(8, {'name': 'John', 'age': 30, 'city': 'New York'})
(9, {'name': 'John', 'age': 30, 'city': 'New York'})
(10, {'name': 'John', 'age': 30, 'city': 'New York'})
(11, {'name': 'John', 'age': 30, 'city': 'New York'})
(12, {'name': 'John', 'age': 30, 'city': 'New York'})
(14, {'name': 'John', 'age': 30, 'city': 'New York'})
(15, {'name': 'John', 'age': 30, 'city': 'New York'})
(17, {'name': 'John', 'age': 30, 'city': 'New York'})
(18, {'name': 'John', 'age': 30, 'city': 'New York'})
(19, {'name': 'John', 'age': 30, 'city': 'New York'})
(20, {'name': 'John', 'age': 30, 'city': 'New York'})
(22, {'name': 'John', 'age': 30, '

## Convert interactive python concepts into more generic function based code

### Database access and creation

In [24]:
from sqlalchemy import text
import pandas as pd

def database_query_function(session, key, value):
    # Query JSON field using a PostgreSQL JSON function
    query = text(f"SELECT * FROM users WHERE data ->> :key = :value")
    result = session.execute(query, {'key': key, 'value': str(value)})

    # Convert result to a pandas DataFrame
    df = pd.DataFrame(result.fetchall())
    df.columns = result.keys()

    session.close()

    # return the DataFrame
    return df

In [26]:
database_query_function(session=session, key='age', value=30)

2024-06-09 09:34:29,004 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-09 09:34:29,004 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE data ->> %(key)s = %(value)s
2024-06-09 09:34:29,005 INFO sqlalchemy.engine.Engine [cached since 0.01089s ago] {'key': 'age', 'value': '30'}
2024-06-09 09:34:29,006 INFO sqlalchemy.engine.Engine ROLLBACK


,id,data
0,1,"{'name': 'John', 'age': 30, 'city': 'New York'}"
1,2,"{'name': 'John', 'age': 30, 'city': 'New York'}"
2,8,"{'name': 'John', 'age': 30, 'city': 'New York'}"
3,9,"{'name': 'John', 'age': 30, 'city': 'New York'}"
4,10,"{'name': 'John', 'age': 30, 'city': 'New York'}"
5,11,"{'name': 'John', 'age': 30, 'city': 'New York'}"
6,12,"{'name': 'John', 'age': 30, 'city': 'New York'}"
7,14,"{'name': 'John', 'age': 30, 'city': 'New York'}"
8,15,"{'name': 'John', 'age': 30, 'city': 'New York'}"
9,17,"{'name': 'John', 'age': 30, 'city': 'New York'}"


### More advanced function

#### Database table creation

In [ ]:
import hashlib
import json
from datetime import datetime
from pathlib import Path

class User:
    def __init__(self, user_id, name, input_dict, path, length=256):
        """
        Initializes a User object with a user ID, name, input dictionary, hash, file path, and creation date.

        The function hashes the input dictionary and stores both the dictionary and its hash, 
        the file path where further data is stored, and the date of creation.

        Parameters:
        user_id (str): The ID of the user.
        name (str): The name of the user.
        input_dict (dict): The dictionary to convert into a hash.
        path (str): The path where further data is stored.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.
        """

        self.user_id = user_id
        self.name = name
        self.input_dict = input_dict
        self.dict_hash = self.compute_dict_hash(input_dict, length=length)
        self.path = Path(path).resolve()
        self.creation_date = datetime.now()

    @staticmethod
    def compute_dict_hash(input_dict, length=256):
        """
        Converts a dictionary into a hash.

        Parameters:
        input_dict (dict): The dictionary to convert into a hash.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.

        Returns:
        str: The hexadecimal representation of the hash of the dictionary.
        """

        # Convert dictionary to JSON object to ensure consistent hash
        jsonified_dict = json.dumps(input_dict, sort_keys=True)

        # Create a new hash object based on specified length
        if length == 256:
            hasher = hashlib.sha256()
        elif length == 512:
            hasher = hashlib.sha512()
        elif length == 128:
            hasher = hashlib.md5()
        else:
            raise ValueError("Length must be either 128, 256 or 512.")

        # Update the hash object with the bytes of the JSON object
        hasher.update(jsonified_dict.encode('utf-8'))

        # Return the hexadecimal representation of the hash
        return hasher.hexdigest()

In [1]:
from sqlalchemy import create_engine, Column, Integer, JSON
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.sql.expression import cast
from sqlalchemy import String
from dataclasses import dataclass

# Declare the Database class
@dataclass
class Database:
    Session: any
    User: any


import hashlib
import json
from datetime import datetime
from pathlib import Path

class User:
    def __init__(self, user_id, name, input_dict, path, length=256):
        """
        Initializes a User object with a user ID, name, input dictionary, hash, file path, and creation date.

        The function hashes the input dictionary and stores both the dictionary and its hash, 
        the file path where further data is stored, and the date of creation.

        Parameters:
        user_id (str): The ID of the user.
        name (str): The name of the user.
        input_dict (dict): The dictionary to convert into a hash.
        path (str): The path where further data is stored.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.
        """

        self.user_id = user_id
        self.name = name
        self.input_dict = input_dict
        self.dict_hash = self.compute_dict_hash(input_dict, length=length)
        self.path = Path(path).resolve()
        self.creation_date = datetime.now()


def create_users_table(db_url='postgresql://localhost/joerg', echo=False):
    # Create engine
    engine = create_engine(db_url, echo=echo)

    # Create a configured "Session" class
    Session = sessionmaker(bind=engine)

    # Declare base
    Base = declarative_base()

    # Define User class
    class User(Base):
        __tablename__ = 'users'

        id = Column(Integer, primary_key=True)
        data = Column(JSON)

    # Create table
    Base.metadata.create_all(bind=engine)
    
    # Return Database class with Session and User classes
    return Database(Session, User)

In [2]:
db = create_users_table()

#### Add data

In [5]:
def add_data_to_database(db, verbose=False, **kwargs):
    """
    Adds a record to the users table in the database.

    Args:
        db (Database): A Database instance containing the Session and User classes.
        verbose (bool): If True, will print a success message. False by default.
        **kwargs: Arbitrary keyword arguments representing the data to be added.

    Returns:
        Database: The original Database instance passed in. 
    """
    # Extract Session and User classes from Database instance
    Session = db.Session
    User = db.User

    # Start a new session
    session = Session()

    # Create a new user with the given key-value data
    new_user = User(data=kwargs)

    # Add the new user to the session and commit
    session.add(new_user)
    session.commit()

    session.close()

    if verbose:
        print(f"Data {kwargs} added to database.")

    return db

In [38]:
data = {'name': 'Alex', 'age': 29, 'city': 'Tübingen'}
add_data_to_database(db, **data, verbose=True)

Data {'name': 'Alex', 'age': 29, 'city': 'Tübingen'} added to database.


Database(Session=sessionmaker(class_='Session', bind=Engine(postgresql://localhost/joerg), autoflush=True, expire_on_commit=True), User=<class '__main__.create_users_table.<locals>.User'>)

#### Delete data

In [36]:
def delete_data_from_database(db, indices=None, verbose=False):
    """
    Deletes selected/named rows in the users table.

    Args:
        db (Database): A Database instance containing the Session and User classes.
        indices (list): Optional. List of indices to delete. If None, deletes all rows.
        verbose (bool): If True, prints out messages about what the function does.

    Returns:
        Database: The original Database instance passed in. 
    """
    # Extract Session and User classes from Database instance
    Session = db.Session
    User = db.User

    # Start a new session
    session = Session()

    ids_to_delete = []

    if indices is None:
        # Delete all rows
        session.query(User).delete()
        message = "All rows deleted."
    else:
        # Check if provided indices exist in the database
        for index in indices:
            user = session.get(User, int(index))  # using session.get instead of query.get
            if user is None:
                if verbose:
                    print(f"Row with id {index} does not exist.")
            else:
                ids_to_delete.append(int(index))

        # Delete only the existing rows with the provided indices
        for index in ids_to_delete:
            session.query(User).filter(User.id == int(index)).delete(synchronize_session='fetch')
            session.expire_all()  # synchronize the session
        message = f"Rows with ids {ids_to_delete} deleted."

    session.commit()

    session.close()

    if verbose:
        print(message)
    
    return db


In [35]:
# db = delete_data_from_database(db, verbose=True)

In [40]:
db = delete_data_from_database(db, indices=[43], verbose=True)

Rows with ids [43] deleted.


#### Show database table as pandas dataframe

In [41]:
import pandas as pd

def fetch_all_data(db):
    """
    Fetches all rows from the users table.

    Args:
        db (Database): A Database instance containing the Session and User classes.

    Returns:
        DataFrame: A pandas DataFrame containing all rows in the users table.
    """
    # Extract Session and User classes from Database instance
    Session = db.Session
    User = db.User

    # Start a new session
    session = Session()

    # Query all rows
    result = session.query(User).all()

    # Convert result to a pandas DataFrame
    df = pd.DataFrame([(user.id, user.data) for user in result], columns=['id', 'data'])

    session.close()

    return df

In [43]:
def transform_data_column(df):
    """
    Transforms the 'data' column of the DataFrame into separate columns.

    Args:
        df (DataFrame): A pandas DataFrame containing the id and data columns.

    Returns:
        DataFrame: A pandas DataFrame where each key in the 'data' column is a separate column.
    """
    # Transform 'data' column into a DataFrame
    data_df = pd.json_normalize(df['data'])

    # Concatenate original DataFrame (minus 'data' column) with the new DataFrame
    df = pd.concat([df.drop('data', axis=1), data_df], axis=1)

    return df

In [44]:
df = fetch_all_data(db)
df = transform_data_column(df)
df

,id,name,age,city
0,44,Alex,29,Tübingen


#### Query

In [3]:
from sqlalchemy import text
import pandas as pd

def database_query_function(db, **kwargs):
    # Extract Session and User classes from Database instance
    Session = db.Session
    User = db.User

    # Start a new session
    session = Session()

    # Convert kwargs values to strings
    kwargs = {k: str(v) for k, v in kwargs.items()}

    # Start building the SQL query
    query_text = "SELECT * FROM users WHERE "

    # Add conditions for each key-value pair
    for key, value in kwargs.items():
        query_text += f"data ->> '{key}' = '{value}' AND "

    # Remove trailing "AND "
    query_text = query_text[:-4]

    # Execute the query
    query = text(query_text)
    result = session.execute(query)

    data = result.fetchall()
    if data:
        # Convert result to a pandas DataFrame
        df = pd.DataFrame(data)
        df.columns = result.keys()
    else:
        df = pd.DataFrame()  # empty DataFrame

    session.close()

    # return the DataFrame
    return df


In [49]:
%%time
df = database_query_function(db=db, age=29, name='Alex')
df

CPU times: user 2.75 ms, sys: 528 µs, total: 3.28 ms
Wall time: 2.91 ms


,id,data
0,44,"{'name': 'Alex', 'age': 29, 'city': 'Tübingen'}"


#### Convert dictionary in hash string

In [58]:
import hashlib
import json
from datetime import datetime
from pathlib import Path

class User:
    def __init__(self, user_id, name, input_dict, path, length=256):
        """
        Initializes a User object with a user ID, name, input dictionary, hash, file path, and creation date.

        The function hashes the input dictionary and stores both the dictionary and its hash, 
        the file path where further data is stored, and the date of creation.

        Parameters:
        user_id (str): The ID of the user.
        name (str): The name of the user.
        input_dict (dict): The dictionary to convert into a hash.
        path (str): The path where further data is stored.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.
        """

        self.user_id = user_id
        self.name = name
        self.input_dict = input_dict
        self.dict_hash = self.compute_dict_hash(input_dict, length=length)
        self.path = Path(path).resolve()
        self.creation_date = datetime.now()

    @staticmethod
    def compute_dict_hash(input_dict, length=256):
        """
        Converts a dictionary into a hash.

        Parameters:
        input_dict (dict): The dictionary to convert into a hash.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.

        Returns:
        str: The hexadecimal representation of the hash of the dictionary.
        """

        # Convert dictionary to JSON object to ensure consistent hash
        jsonified_dict = json.dumps(input_dict, sort_keys=True)

        # Create a new hash object based on specified length
        if length == 256:
            hasher = hashlib.sha256()
        elif length == 512:
            hasher = hashlib.sha512()
        elif length == 128:
            hasher = hashlib.md5()
        else:
            raise ValueError("Length must be either 128, 256 or 512.")

        # Update the hash object with the bytes of the JSON object
        hasher.update(jsonified_dict.encode('utf-8'))

        # Return the hexadecimal representation of the hash
        return hasher.hexdigest()


In [92]:
import hashlib
import json

def compute_dict_hash(input_dict, length=256):
    """
    Converts a dictionary into a hash.
    
    This function converts a dictionary into a JSON object to ensure a 
    consistent hash and hashes it using SHA-256 by default but can be 
    adjusted for SHA-512 or MD5.

    Parameters:
    input_dict (dict): The dictionary to convert into a hash.
    length (int, optional): The desired length of the hash, either 
                             128, 256 (default), or 512.

    Returns:
    str: The hexadecimal representation of the hash of the dictionary,
         matching the specified length.
    """
    
    # Convert dictionary to JSON object to ensure consistent hash
    jsonified_dict = json.dumps(input_dict, sort_keys=True)

    # Create a new hash object based on specified length
    if length == 256:
        hasher = hashlib.sha256()
    elif length == 512:
        hasher = hashlib.sha512()
    elif length == 128:
        hasher = hashlib.md5()
    else:
        raise ValueError("Length must be either 128, 256, or 512.")

    # Update the hash object with the bytes of the JSON object
    hasher.update(jsonified_dict.encode('utf-8'))

    # Return the hexadecimal representation of the hash
    return hasher.hexdigest()


In [57]:
data_dict = df.data.iloc[0]
dict_hash(data_dict, length=512)

'74b7e9a15ecbb49921e9a328005723513f0cd7b1d5c7efb39ac87fe9aafc0fb5d8f1d14b5da58d12047efa0a72a1eb837a4e1a61585eb18c4032c0940e47da80'

### Create table class and database

In [86]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, JSON
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
from pathlib import Path

Base = declarative_base()

class Node(Base):
    __tablename__ = 'node'

    node_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    dict_hash = Column(String)
    path = Column(String)
    creation_date = Column(DateTime, default=datetime.utcnow)
    data = Column(JSON)

    def __init__(self, name, input_dict, path, length=256):
        """
        Initializes a Node object with a name, input dictionary, hash, file path, 
        and creation date. 

        The function hashes the input dictionary and stores both the dictionary and its hash,  
        and the file path where further data is stored. The `node_id` and `creation_date`
        are auto-generated.

        Parameters:
        name (str): The name of the node.
        input_dict (dict): The dictionary to convert into a hash.
        path (str): The path where further data is stored.
        length (int, optional): The desired length of the hash, either 128, 256 (default), or 512.
        """

        self.name = name
        self.dict_hash = compute_dict_hash(input_dict, length=length)
        self.path = str(Path(path).resolve())
        self.data = input_dict


In [87]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dataclasses import dataclass

@dataclass
class Database:
    """
    Data class that represents a Database.

    Attributes:
    Session (sqlalchemy.orm.sessionmaker): The sessionmaker object that creates session.
    User (Base): The SQLAlchemy Base object that represents a user.
    """
    Session: any
    Node: any

def create_users_table(db_url='postgresql://localhost/joerg', echo=False):
    """
    Creates a users table in the database.
    
    This function creates a SQLAlchemy engine with the provided database URL,
    sets up a sessionmaker with this engine, ensures the users table is 
    created, and returns a Database object with the sessionmaker and the User class.

    Parameters:
    db_url (str, optional): The database URL. Defaults to 'postgresql://localhost/joerg'.
    echo (bool, optional): If true, the engine will log all statements 
                           as well as a repr() of their parameter lists to the engines logger. 
                           Defaults to False.

    Returns:
    Database: a Database object with two attributes: a configured "Session" class and the "User" class.
    """
    
    # Create engine
    engine = create_engine(db_url, echo=echo)

    # Create a configured "Session" class
    Session = sessionmaker(bind=engine)

    # Create table
    Node.__table__.create(bind=engine, checkfirst=True)

    # Return Database class with Session and Node classes
    return Database(Session, Node)

In [88]:
db = create_users_table()

#### Add node

In [89]:
from sqlalchemy.orm import Session

def add_node_to_db(db, data, name='', path='.', length=256):
    """
    Adds a dictionary as a new Node to a database.

    Parameters:
    db (Database): The Database object, which includes the Session and Node classes.
    name (str): The name of the Node.
    data (dict): The dictionary to convert into a hash and store as data.
    path (str): The path where further data is stored.
    length (int): The desired length of the hash.
    

    Returns: 
    Node:  The Node object that was added to the database.
    """
    
    # Start session
    session = db.Session()

    # Create new node, without providing node_id
    new_node = db.Node(name=name, input_dict=data, path=path, length=length)
    
    # Add new node to session
    session.add(new_node)
    
    # Commit session
    session.commit()

    # Close session
    session.close()

    return new_node

In [90]:
data

{'name': 'Alex', 'age': 29, 'city': 'Tübingen'}

In [93]:
add_node_to_db(db=db, data=data)

#### Show database table as dataframe

In [94]:
import pandas as pd
from sqlalchemy.orm import Session

def get_db_table_as_df(db):
    """
    Get all data in the 'node' database table as a pandas DataFrame.

    Parameters:
    db (Database): The Database object, which includes the Session and Node classes.

    Returns:
    DataFrame: The entire 'node' database table as a pandas DataFrame.
    """
    
    # Start session
    session = db.Session()

    # Run SELECT * FROM node
    df = pd.read_sql(session.query(db.Node).statement, session.bind)

    return df


In [95]:
get_db_table_as_df(db)

,node_id,name,dict_hash,path,creation_date,data
0,1,,a24884d4e22a4fe7ffc5aa89348529785a71befbf3aa8a...,/Users/joerg/python_projects/git_libs/pyiron_n...,2024-06-09 12:13:34.632183,"{'name': 'Alex', 'age': 29, 'city': 'Tübingen'}"
